# ClimateHack.AI 2023: Data Exploration

Thank you for participating in ClimateHack.AI 2023! 🌍

Your contributions could help cut carbon emissions by up to 100 kilotonnes per year in Great Britain alone. We look forward to seeing what you build over the course of the competition!

As with any machine learning task, the best place to start is by inspecting the data available, and for this competition, we are spoiled for choice! In total, 600 GB of (mostly compressed) training data is available for you to use and experiment with, including 73 GB of [HRV satellite imagery](https://huggingface.co/datasets/climatehackai/climatehackai-2023/tree/main/satellite-hrv), 189 GB of [non-HRV satellite imagery](https://huggingface.co/datasets/climatehackai/climatehackai-2023/tree/main/satellite-nonhrv), 78 GB of [weather forecasts](https://huggingface.co/datasets/climatehackai/climatehackai-2023/tree/main/weather), 259 GB of [air quality forecasts](https://huggingface.co/datasets/climatehackai/climatehackai-2023/tree/main/aerosols) and ~1 GB of [historical solar PV generation data](https://huggingface.co/datasets/climatehackai/climatehackai-2023/tree/main/pv).

You do not have you use all of the data for this challenge (and in fact, you probably shouldn't lest you end up with a bloated model!). Having said that, it is up to you to **be creative** to decide which data sources you actually do want to use and train on! Ultimately, you want to build an **accurate and performant machine learning model** capable of extracting as much useful information from the available data as possible in order to maximally explain any variance in solar PV yields.

For more detailed information on the available data, see the [DOXA AI competition page](https://doxaai.com/competition/climatehackai-2023/overview). 😎

## Prerequisites

If you do not have the following Python packages installed, you can uncomment and run the following line to install them with `pip`. 

In [ ]:
# %pip install numpy matplotlib zarr xarray ipykernel gcsfs fsspec dask cartopy ocf-blosc2
# %pip install -U doxa-cli

## Importing packages

In [ ]:
import json
import os

import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
from ocf_blosc2 import Blosc2

plt.rcParams["figure.figsize"] = (20, 12)

## Working with Zarr datasets

Most of the data for this competition uses the [Zarr storage format](https://zarr.dev/), which allows for incredibly large NumPy-like multi-dimensional arrays to be split and stored as individually compressed chunks in a relatively efficient way while still letting you interact with the data as if it were one contiguous whole. This means that you do not in theory have to load all of the data you wish to use into memory at once.

One benefit of the Zarr format is that Zarr datasets can be streamed straight from the cloud. While this most likely will not be fast enough in training, it already lets us perform some initial data exploration without having to download entire months of data.

In Python, we can work with Zarr stores using [xarray](https://docs.xarray.dev/en/stable/user-guide/io.html#zarr), as we will see!

## HRV Satellite Imagery

We can load the high-resolution visible (HRV) EUMETSAT satellite imagery data straight from [HuggingFace](https://huggingface.co/datasets/climatehackai/climatehackai-2023/tree/main/satellite-hrv). The satellite imagery covers an area slightly larger than Great Britain and is taken every five minutes. Since this data has a much higher resolution than the non-HRV satellite imagery data, clouds are a lot more visible, which may be of particular interest to you.

In [ ]:
hrv = xr.open_dataset(
    "zip:///::https://huggingface.co/datasets/climatehackai/climatehackai-2023/resolve/main/satellite-hrv/2020/7.zarr.zip",
    engine="zarr",
    consolidated=True,
)

hrv

We can use the `.plot()` method to take a look at what the HRV data looks like at a particular moment in time.

In [ ]:
hrv["data"].sel(time="2020-07-20 10:00").plot()  # type: ignore

A slightly more advanced version of this allows us to draw coastlines on top of the data.

In [ ]:
axes = plt.axes(projection=ccrs.Geostationary(central_longitude=9.5))

hrv["data"].sel(time="2020-07-20 10:00", channel="HRV").plot.pcolormesh(
    ax=axes,
    transform=ccrs.Geostationary(central_longitude=9.5),
    x="x_geostationary",
    y="y_geostationary",
    add_colorbar=False,
)  # type: ignore

axes.coastlines()

As part of this challenge, you are given 128x128 satellite imagery crops centred on each solar PV site taken over the previous hour.

In [ ]:
fig, ax = plt.subplots(1, 12, figsize=(15, 3))

for j, time in enumerate(
    [
        "2020-07-04 12:00",
        "2020-07-04 12:05",
        "2020-07-04 12:10",
        "2020-07-04 12:15",
        "2020-07-04 12:20",
        "2020-07-04 12:25",
        "2020-07-04 12:30",
        "2020-07-04 12:35",
        "2020-07-04 12:40",
        "2020-07-04 12:45",
        "2020-07-04 12:50",
        "2020-07-04 12:55",
    ]
):
    ax[j].imshow(
        hrv["data"]
        .sel(time=time)
        .isel(x_geostationary=slice(128, 256), y_geostationary=slice(128, 256))
        .to_numpy(),
        cmap="viridis",
    )
    ax[j].get_xaxis().set_visible(False)
    ax[j].get_yaxis().set_visible(False)

## Non-HRV Satellite Imagery

We can also perform something similar for the non-HRV satellite imagery data.

In [ ]:
nonhrv = xr.open_dataset(
    "zip:///::https://huggingface.co/datasets/climatehackai/climatehackai-2023/resolve/main/satellite-nonhrv/2020/7.zarr.zip",
    engine="zarr",
    consolidated=True,
)

nonhrv

Notice how the non-HRV satellite imagery data is composed of 11 different channels:

In [ ]:
nonhrv.channel

These 11 non-HRV channels correspond to different wavelengths and types of satellite imagery (visible, infrared and water vapour).

We can select one of these channels (in this case, an infrared one) and plot it in a similar way to the previous example involving HRV data.

In [ ]:
nonhrv["data"].sel(time="2020-07-20 10:00", channel="IR_016").plot()  # type: ignore

As with the HRV data, you are given 128x128 crops for all 11 non-HRV channels centred on each solar PV site taken over the previous hour. Just note that the non-HRV data, having a lower resolution, corresponds to a much larger area.

In [ ]:
fig, axes = plt.subplots(12, len(nonhrv.channel), figsize=(16, 18))

for ax, channel in zip(axes[0], nonhrv.channel):
    ax.set_title(channel.item(), rotation=0, size="large")

for i, time in enumerate(
    [
        "2020-07-04 12:00",
        "2020-07-04 12:05",
        "2020-07-04 12:10",
        "2020-07-04 12:15",
        "2020-07-04 12:20",
        "2020-07-04 12:25",
        "2020-07-04 12:30",
        "2020-07-04 12:35",
        "2020-07-04 12:40",
        "2020-07-04 12:45",
        "2020-07-04 12:50",
        "2020-07-04 12:55",
    ]
):
    for j, channel in enumerate(nonhrv.channel):
        axes[i][j].imshow(
            nonhrv["data"]
            .sel(time=time, channel=channel)
            .isel(x_geostationary=slice(128, 256), y_geostationary=slice(128, 256))
            .to_numpy(),
            cmap="viridis",
        )
        axes[i][j].get_xaxis().set_visible(False)
        axes[i][j].get_yaxis().set_visible(False)

As an aside, whenever you read a slice of data from a Zarr data array, all the chunks corresponding to your selection need to be individually fetched and decompressed. As an example, the non-HRV data, which has dimensions `(time, y_geostationary, x_geostationary, channel)`, is stored as chunks with the shape `(1, 293, 333, 11)`, meaning that each chunk contains the data for all 11 non-HRV channels at a single time step. 

Especially when you account of the chunk caching that takes place under the hood, this means that it is significantly faster to select multiple non-HRV channels for the same time step together (since they are all stored within the same chunk) than it is to select multiple time steps' worth of data for each channel separately. You can see this effect by swapping the order of the loops in the code block above.

## Weather Forecasts

We can also look at the weather forecast dataset by loading and visualising it in a very similar way!

As you can see, this dataset is composed of 38 different data variables (many of which correspond to different altitudes), such as for ground temperatures, total precipitation and more. Forecasts are available on an hourly basis. For further information on each of these data variables, check out the data section on the [ClimateHack.AI 2023 competition page](https://doxaai.com/competition/climatehackai-2023/overview).

In [ ]:
nwp = xr.open_dataset(
    "zip:///::https://huggingface.co/datasets/climatehackai/climatehackai-2023/resolve/main/weather/2020/7.zarr.zip",
    engine="zarr",
    consolidated=True,
)

nwp

### Ground temperatures

Just as with the satellite imagery data, we can also plot individual data variables in the weather forecast dataset. Here, `t_g` corresponds to ground-level temperatures in Kelvin (which we convert to Celsius in the visualisation below).

In [ ]:
axes = plt.axes(projection=ccrs.PlateCarree())

(nwp["t_g"].sel(time="2020-07-20 10:00") - 273.15).plot.pcolormesh(
    ax=axes,
    transform=ccrs.PlateCarree(),
    x="longitude",
    y="latitude",
    add_colorbar=True,
    cmap="coolwarm",
)  # type: ignore

axes.coastlines()

Just as with the satellite imagery, we can generate 128x128 crops for each weather forecast data variable as well. As part of this challenge, you are given six time steps' worth of weather forecast data, corresponding to the beginnings of the previous hour, the current hour and the next four hours.

In [ ]:
fig, ax = plt.subplots(1, 6, figsize=(15, 3))

for j, time in enumerate(
    [
        "2020-07-04 10:00",
        "2020-07-04 11:00",
        "2020-07-04 12:00",
        "2020-07-04 13:00",
        "2020-07-04 14:00",
        "2020-07-04 15:00",
    ]
):
    ax[j].imshow(
        nwp["t_g"]
        .sel(time=time)
        .isel(latitude=slice(100, 228), longitude=slice(100, 228))
        .to_numpy(),
        cmap="coolwarm",
    )
    ax[j].get_xaxis().set_visible(False)
    ax[j].get_yaxis().set_visible(False)

### Cloud cover

Similarly, we can also look at total cloud cover forecasts (`clct`).

In [ ]:
axes = plt.axes(projection=ccrs.PlateCarree())

(nwp["clct"].sel(time="2020-07-20 10:00") - 273.15).plot.pcolormesh(
    ax=axes,
    transform=ccrs.PlateCarree(),
    x="longitude",
    y="latitude",
    add_colorbar=True,
)  # type: ignore

axes.coastlines()

## All weather variables

Here are all the weather variables available in this dataset.

In [ ]:
nrows = 8
ncols = 5

fig, axes = plt.subplots(
    nrows=nrows,
    ncols=ncols,
    figsize=(10, 20),
    subplot_kw={"projection": ccrs.PlateCarree()},
)

for j, var in enumerate(nwp.data_vars):
    nwp[var].sel(time="2020-07-20 10:00",).plot.pcolormesh(
        ax=axes[j // ncols][j % ncols],
        transform=ccrs.PlateCarree(),
        x="longitude",
        y="latitude",
        add_colorbar=False,
        cmap="coolwarm" if var.split("_")[0] in ("t", "v", "u") else "viridis",
    )

    axes[j // ncols][j % ncols].coastlines()
    axes[j // ncols][j % ncols].get_xaxis().set_visible(False)
    axes[j // ncols][j % ncols].get_yaxis().set_visible(False)
    axes[j // ncols][j % ncols].set_title(var)

fig.tight_layout()
fig.subplots_adjust(wspace=0.1, hspace=0.1)

As part of the challenge, you get to be really creative with what data variables you choose to use and integrate into your model. Not all of them will necessarily be relevant to you, but here are some interesting ones to potentially look at first:

- `alb_rad`: Surface albedo (%)
- `clch`: Cloud cover at high levels (0-400 hPa) (%)
- `clcl`: Cloud cover at low levels (800 hPa - soil) (%)
- `clcm`: Cloud cover at mid levels (400-800 hPa) (%)
- `clct`: Total cloud cover (%)
- `t`: Air temperature (K) – available for three different pressure levels (at 5600m, 1500m and 500m)

## Air Quality Forecasts

Finally, we can also explore the ECMWF CAMS air quality forecast dataset, which contains a number of data variables related to aerosols in the atmosphere at 8 different levels. There is a lot of aerosol data available, so if you are interested in using the aerosol data as part of your submission, it is worth spending some time to get familiar with the data and figure out which data variables may actually be useful to you. For example, not all aerosol types are found in large concentrations over Great Britain, which is our area of interest. 

In [ ]:
aerosols = xr.open_dataset(
    "zip:///::https://huggingface.co/datasets/climatehackai/climatehackai-2023/resolve/main/aerosols/2020/7.zarr.zip",
    engine="zarr",
    consolidated=True,
)

aerosols

You can access a list of available data variables programmatically through the `data_vars` attribute.

In [ ]:
aerosols.data_vars

You can also inspect the different levels (or altitudes) for which the aerosol forecasts are available for each data variable.

In [ ]:
aerosols.level

We can also visualise the aerosol forecasts in a similar way to the satellite imagery and weather forecasts.

In [ ]:
axes = plt.axes(projection=ccrs.PlateCarree())

aerosols["pm10_conc"].sel(time="2020-07-20 10:00", level=1000).plot.pcolormesh(
    ax=axes,
    transform=ccrs.PlateCarree(),
    x="longitude",
    y="latitude",
    add_colorbar=True,
)  # type: ignore

axes.coastlines()

In [ ]:
fig, axes = plt.subplots(
    nrows=len(aerosols.data_vars),
    ncols=len(aerosols.level),
    figsize=(15, 28),
    subplot_kw={"projection": ccrs.PlateCarree()},
)

for j, var in enumerate(aerosols.data_vars):
    for i, level in enumerate(aerosols.level):
        aerosols[var].sel(time="2020-07-20 10:00", level=level).plot.pcolormesh(
            ax=axes[j][i],
            transform=ccrs.PlateCarree(),
            x="longitude",
            y="latitude",
            add_colorbar=False,
            cmap="viridis",
        )

        axes[j][i].coastlines()
        axes[j][i].get_xaxis().set_visible(False)
        axes[j][i].get_yaxis().set_visible(False)
        axes[j][i].set_title(f"{var} ({int(level)}m)")

fig.tight_layout()
fig.subplots_adjust(wspace=0.1, hspace=0.1)

We can generate 128x128 crops of the air quality forecast data in a similar way to the weather data, with the main difference being that the air quality forecasts have an additional `level` dimension. Otherwise, for this challenge, you are given data for the same six time steps as the weather forecast data, namely forecasts associated with the beginnings of the previous hour, the current hour and the next four hours.

In [ ]:
fig, axes = plt.subplots(6, len(aerosols.level), figsize=(16, 12))

for ax, channel in zip(axes[0], aerosols.level):
    ax.set_title(channel.item(), rotation=0, size="large")

for i, time in enumerate(
    [
        "2020-07-04 10:00",
        "2020-07-04 11:00",
        "2020-07-04 12:00",
        "2020-07-04 13:00",
        "2020-07-04 14:00",
        "2020-07-04 15:00",
    ]
):
    for j, level in enumerate(aerosols.level):
        axes[i][j].imshow(
            aerosols["pm2p5_conc"]
            .sel(time=time, level=level)
            .isel(latitude=slice(100, 228), longitude=slice(100, 228))
            .to_numpy(),
            cmap="viridis",
        )
        axes[i][j].get_xaxis().set_visible(False)
        axes[i][j].get_yaxis().set_visible(False)

## Next steps

Now that you have had an overview of the data available, you are now ready to train your first model and make your first submission for ClimateHack.AI 2023! 🥳

See the Jupyter notebook `2_training.ipynb` for the next part of these getting started materials.